In [18]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('dataset/train')

label_types = os.listdir('dataset/train')
print (label_types)  

['Clapping', 'Running']


In [19]:

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('dataset/train' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('dataset/train' + '/' +item) + '/' + room))
    
# Build a dataframe        
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

        tag                               video_name
0  Clapping  dataset/train/Clapping/Clapping_500.mp4
1   Running    dataset/train/Running/Running_500.mp4
        tag                               video_name
0  Clapping  dataset/train/Clapping/Clapping_500.mp4
1   Running    dataset/train/Running/Running_500.mp4


In [20]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

In [21]:
dataset_path = os.listdir('dataset/test')
print(dataset_path)

room_types = os.listdir('dataset/test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('dataset/test' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('dataset/test' + '/' +item) + '/' + room))
    
# Build a dataframe        
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['Clapping', 'Running']
Types of activities found:  2
        tag                                 video_name
0  Clapping     dataset/test/Clapping/Clapping_500.mp4
1   Running  dataset/test/Running/Running_500 copy.mp4
        tag                                 video_name
0  Clapping     dataset/test/Clapping/Clapping_500.mp4
1   Running  dataset/test/Running/Running_500 copy.mp4


In [22]:
!pip install git+https://github.com/tensorflow/docs

^C


  Cloning https://github.com/tensorflow/docs to c:\users\anomi\appdata\local\temp\pip-req-build-9eon7vxd
  Resolved https://github.com/tensorflow/docs to commit c221d1e1af1ef5cc37c4a0879876f25f9cc1d981
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs 'C:\Users\anomi\AppData\Local\Temp\pip-req-build-9eon7vxd'

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:

from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [14]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

In [24]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(1)

Total videos for training: 2
Total videos for testing: 2


,Unnamed: 0,video_name,tag
1,1,dataset/train/Running/Running_500.mp4,Running


In [26]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [27]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 195s 2us/step


In [28]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['Clapping', 'Running']


array([[0],
       [1]], dtype=int64)

In [29]:
#print(train_data[0].shape)
#train_data[0]

In [30]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [ ]:
with tf.device('/CPU:0'):
        frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
        mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

        # Refer to the following tutorial to understand the significance of using `mask`:
        # https://keras.io/api/layers/recurrent_layers/gru/
        # Use the CPU implementation of GRU by setting recurrent_activation='sigmoid'
        # and reset_after=True if you encounter CuDNN errors.
        x = keras.layers.GRU(16, return_sequences=True, recurrent_activation='sigmoid', reset_after=False)(
            frame_features_input, mask=mask_input
        )
        x = keras.layers.GRU(8, recurrent_activation='sigmoid', reset_after=True)(x)
        x = keras.layers.Dropout(0.4)(x)
        x = keras.layers.Dense(224, activation="leaky_relu")(x)
        x = keras.layers.Dense(56, activation="leaky_relu")(x)
        x = keras.layers.Dense(14, activation="leaky_relu")(x)
        x = keras.layers.Dense(8, activation="leaky_relu")(x)
        output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

In [ ]:
from tensorflow import keras

def create_cnn_rnn_model(MAX_SEQ_LENGTH, NUM_FEATURES, class_vocab):
    with tf.device('/CPU:0'):
        # Input layers
        frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
        mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

        # Reshape input for convolution (adding a channel dimension)
        reshaped_input = keras.layers.Reshape((MAX_SEQ_LENGTH, NUM_FEATURES, 1))(frame_features_input)

        # Convolutional layers to capture spatial features
        x = keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(reshaped_input)
        x = keras.layers.MaxPooling2D((2, 2))(x)

        x = keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
        x = keras.layers.MaxPooling2D((2, 2))(x)

        # Flatten spatial features for temporal processing
        x = keras.layers.TimeDistributed(keras.layers.Flatten())(x)

        # GRU layers for temporal modeling
        x = keras.layers.GRU(16, return_sequences=True, recurrent_activation='sigmoid', reset_after=False)(
            x, mask=mask_input
        )
        x = keras.layers.GRU(8, recurrent_activation='sigmoid', reset_after=True)(x)

        # Fully connected layers
        x = keras.layers.Dropout(0.4)(x)
        x = keras.layers.Dense(224, activation="relu")(x)
        x = keras.layers.Dense(56, activation="relu")(x)
        x = keras.layers.Dense(14, activation="relu")(x)
        x = keras.layers.Dense(8, activation="relu")(x)

        # Output layer
        output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

        # Compile the model
        rnn_model = keras.Model([frame_features_input, mask_input], output)
        rnn_model.compile(
            loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
        )

        return rnn_model


In [ ]:
with tf.device('/CPU:0'):
        # Input layers
        frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
        mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

        # Reshape input for convolution (adding a channel dimension)
        reshaped_input = keras.layers.Reshape((MAX_SEQ_LENGTH, NUM_FEATURES, 1))(frame_features_input)

        # Convolutional layers to capture spatial features
        x = keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(reshaped_input)
        x = keras.layers.MaxPooling2D((2, 2))(x)

        x = keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
        x = keras.layers.MaxPooling2D((2, 2))(x)

        # Flatten spatial features for temporal processing
        x = keras.layers.TimeDistributed(keras.layers.Flatten())(x)

        # GRU layers for temporal modeling
        x = keras.layers.GRU(16, return_sequences=True, recurrent_activation='sigmoid', reset_after=False)(
            x, mask=mask_input
        )
        x = keras.layers.GRU(8, recurrent_activation='sigmoid', reset_after=True)(x)

        # Fully connected layers
        x = keras.layers.Dropout(0.4)(x)
        x = keras.layers.Dense(224, activation="relu")(x)
        x = keras.layers.Dense(56, activation="relu")(x)
        x = keras.layers.Dense(14, activation="relu")(x)
        x = keras.layers.Dense(8, activation="relu")(x)

        # Output layer
        output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

        # Compile the model
        rnn_model = keras.Model([frame_features_input, mask_input], output)
        rnn_model.compile(
            loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
        )

        return rnn_model